## NOTE: This book doesn't work

In [31]:
import os
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder

from sklearn.cross_validation import cross_val_score

from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

In [2]:
data_dir = os.path.join(os.path.expanduser('~'), 'workspace/talkinData/data')
sub_dir = os.path.join(os.path.expanduser('~'), 'workspace/talkinData/sub')

In [3]:
train = pd.read_csv(os.path.join(data_dir, 'gender_age_train.csv'))

pbdm = pd.read_csv(os.path.join(data_dir, 'phone_brand_device_model.csv'))
pbdm.drop_duplicates('device_id', keep='first', inplace=True)

test = pd.read_csv(os.path.join(data_dir, 'gender_age_test.csv'))

sample_sub = pd.read_csv(os.path.join(sub_dir, 'sample_submission.csv'))

In [4]:
train.head()

,device_id,gender,age,group
0,-8076087639492063270,M,35,M32-38
1,-2897161552818060146,M,35,M32-38
2,-8260683887967679142,M,35,M32-38
3,-4938849341048082022,M,30,M29-31
4,245133531816851882,M,30,M29-31


In [5]:
pbdm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 186716 entries, 0 to 187243
Data columns (total 3 columns):
device_id       186716 non-null int64
phone_brand     186716 non-null object
device_model    186716 non-null object
dtypes: int64(1), object(2)
memory usage: 5.7+ MB


In [6]:
train_all = pd.merge(train, pbdm, on='device_id', how='left', left_index=True)

test_all = pd.merge(test, pbdm, on='device_id', how='left', left_index=True)

In [7]:
train_all.head()

,device_id,gender,age,group,phone_brand,device_model
56800,-8076087639492063270,M,35,M32-38,小米,MI 2
41294,-2897161552818060146,M,35,M32-38,小米,MI 2
9993,-8260683887967679142,M,35,M32-38,小米,MI 2
179893,-4938849341048082022,M,30,M29-31,小米,红米note
102570,245133531816851882,M,30,M29-31,小米,MI 3


In [8]:
cat_col = ['phone_brand', 'device_model']

lb = LabelEncoder()
for var in cat_col:
    full_data = pd.concat((train_all[var], test_all[var]), axis=0).astype('str')
    lb.fit(full_data)
    train_all[var] = lb.transform(train_all[var].astype('str'))
    test_all[var] = lb.transform(test_all[var].astype('str'))

In [9]:
data_x = train_all.ix[:, ['phone_brand', 'device_model']].values
data_y_class = train_all.gender.values
data_y_regress = train_all.age.values

data_x_test = test_all.ix[:, ['phone_brand', 'device_model']].values

In [12]:
data_x_test.shape

(112071, 2)

In [26]:
clf1 = RandomForestClassifier(n_estimators=250, n_jobs=3, max_depth=7)

In [23]:
clf2 = LogisticRegression(n_jobs=3)

In [27]:
cross_val_score(clf1, data_x, data_y_class, cv = 3, scoring='log_loss')

array([-0.64213471, -0.64413321, -0.64612098])

In [25]:
cross_val_score(clf2, data_x, data_y_class, cv = 3, scoring='log_loss')

array([-0.6491995 , -0.65021296, -0.65150877])

In [19]:
clf1 = RandomForestRegressor(n_estimators=250, n_jobs=3, oob_score=True)

In [29]:
clf2 = LinearRegression(n_jobs=3)

In [38]:
clf3 = Lasso(alpha=10)

In [20]:
cross_val_score(clf1, data_x, data_y_regress, cv = 3, scoring='mean_squared_error')

array([-122.04013461,  -90.64835668,  -82.86933481])

In [30]:
cross_val_score(clf2, data_x, data_y_regress, cv = 3, scoring='mean_squared_error')

array([-122.38571588,  -89.55199064,  -82.91371167])

In [39]:
cross_val_score(clf3, data_x, data_y_regress, cv = 3, scoring='mean_squared_error')

array([-122.51639685,  -89.57629976,  -82.82974724])

In [42]:
a = pd.DataFrame({'Gender':data_y_class})

In [45]:
b = pd.get_dummies(a)

In [46]:
b.head()

,Gender_F,Gender_M
0,0.0,1.0
1,0.0,1.0
2,0.0,1.0
3,0.0,1.0
4,0.0,1.0


In [105]:
clf1.classes_

array(['F23-', 'F24-26', 'F27-28', 'F29-32', 'F33-42', 'F43+', 'M22-',
       'M23-26', 'M27-28', 'M29-31', 'M32-38', 'M39+'], dtype=object)

In [55]:
22-, ..., 43+

def binner(var):
    if var['gender'] == 'M':
        if var['age'] <= 22:
            return 'M22-'
        elif var['age'] > 22 and var['age'] < 27:
            return 'M23-26'
        elif var['age'] == 27 or var['age']  == 28:
            return 'M27-28'
        elif var['age'] >= 29 and var['age']  <= 31:
            return 'M29-31'
        elif var['age'] >= 32 and var['age']  <= 38:
            return 'M32-38'
        else:
            return 'M39+'
    else:
        if var['age'] <= 23:
            return 'F23-'
        elif var['age'] > 23 and var['age'] < 27:
            return 'F24-26'
        elif var['age'] == 27 or var['age']  == 28:
            return 'F27-28'
        elif var['age'] >= 29 and var['age']  <= 32:
            return 'F29-32'
        elif var['age'] >= 33 and var['age']  <= 42:
            return 'F33-42'
        else:
            return 'F43+'

c = train.apply(binner, axis=1)

In [70]:
print 'Train:\n', train.head(), '\n\n\n','new_group:\n',  c.head()

Train:
             device_id gender  age   group
0 -8076087639492063270      M   35  M32-38
1 -2897161552818060146      M   35  M32-38
2 -8260683887967679142      M   35  M32-38
3 -4938849341048082022      M   30  M29-31
4   245133531816851882      M   30  M29-31 


new_group:
0    M32-38
1    M32-38
2    M32-38
3    M29-31
4    M29-31
dtype: object


In [66]:
train.group.equals(c)

True